In [4]:
import pandas

# Importing data and cleaning it
strikesDf = pandas.read_csv('data/strike-incidence.csv')
inflationDf = pandas.read_csv('data/inflation.csv')
unemploymentDf = pandas.read_csv('data/unemployment.csv')
labourShareDf = pandas.read_csv('data/labour-share-of-income.csv')
realGdpDf = pandas.read_csv('data/real-gdp.csv')